In [1]:
from copy import deepcopy
import numpy as np
import pandas as pd
import calendar
from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus import AbstractRetrieval

from tqdm import tqdm

pd.options.display.max_columns = 30

filename = "./energy_RE.txt"

In [2]:
### language
dic_language = {'eng': "English",
                'kor': "Korean"
               }


In [3]:
df_re = pd.read_pickle("./df_re.pkl")
print(df_re.shape)
df_re.head()

(23805, 34)


,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,affiliation_city,affiliation_country,author_count,author_names,author_ids,...,issn,source_id,eIssn,aggregationType,volume,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor
0,2-s2.0-85109755005,10.1016/j.ijepes.2021.107370,S0142061521006098,None,Generation maintenance scheduling with renewab...,ar,Article,Assis F.A.,60032361;60028351,Pontifícia Universidade Católica do Rio de Jan...,Rio de Janeiro;Sao Joao del Rei,Brazil;Brazil,5,"Assis, Fernando A.;Leite da Silva, Armando M.;...",57193615355;7003433887;55662238100;56045547600...,...,01420615,17985,None,Journal,134,None,107370,None,This work proposes an efficient method for sol...,Generation maintenance scheduling | Optimizati...,0,0,CAPES,undefined,Coordenação de Aperfeiçoamento de Pessoal de N...
1,2-s2.0-85109460674,10.1016/j.aej.2021.06.026,S1110016821003872,None,A review on segregation of various high gain c...,re,Review,Divya Navamani J.,60015723;60014340,Prince Sultan University;SRM Institute of Scie...,Riyadh;Chennai,Saudi Arabia;India,4,"Divya Navamani, J.;Lavanya, A.;Almakhles, Dhaf...",56879171200;56230077300;56663152700;57222011515,...,11100168,13907,None,Journal,61,1,None,675-700,Renewable powered dc microgrid is more cost-ef...,Boost converters | High gain DC/DC converter |...,0,1,PSU,undefined,Prince Sultan University
2,2-s2.0-85109432210,10.1016/j.aej.2021.06.028,S1110016821003902,None,Application of polynomial regression and MILP ...,ar,Article,Sarwar S.,60029157;60014982;60008665;60004351,Universiti Malaya;National Cheng Kung Universi...,Kuala Lumpur;Tainan;Al Ain;Shah Alam,Malaysia;Taiwan;United Arab Emirates;Malaysia,8,"Sarwar, Sohail;Mokhlis, Hazlie;Othman, Mohamad...",57218645464;8136874200;35975220900;57189691198...,...,11100168,13907,None,Journal,61,1,None,659-674,"Distributed Generation (DG) integration, espec...",Frequency instability | Load priority | Mixed ...,0,1,UM,IIRG001A-2020IISS,Universiti Malaya
3,2-s2.0-85108730066,10.1016/j.ces.2021.116871,S000925092100436X,None,CO<inf>2</inf> methanation in a shell and tube...,ar,Article,Di Nardo A.,60003686,ENEA Centro Ricerche Casaccia,Rome,Italy,4,"Di Nardo, Antonio;Calchetti, Giorgio;Bassano, ...",16318341400;57193360613;55775494000;13102724500,...,00092509,16413,None,Journal,246,None,116871,None,CO2 methanation is gaining increasing interest...,CFD multi-tubular reactor modelling | CO meth...,0,0,ENEA,undefined,European Neuroendocrine Association
4,2-s2.0-85108598105,10.1016/j.ces.2021.116862,S0009250921004279,None,Reverse traveling microwave reactor – Modellin...,ar,Article,Eghbal Sarabi F.,60118243,TU Delft - Department of Process &amp; Energy,Delft,Netherlands,4,"Eghbal Sarabi, Farnaz;Liu, Jiawen;Stankiewicz,...",57212132097;57224936698;7103189558;56734475200,...,00092509,16413,None,Journal,246,None,116862,None,Microwave heating presents a potentially green...,Microwave heating | Reverse traveling microwav...,0,1,H2020,680777,Horizon 2020 Framework Programme


In [4]:
columns=["eid", "PT", "AU", "AF", "TI", 
         "SO", "SO_abb", "LA", "DT", "DE", 
         "ID", "AB", "C1", "RP", "EM", 
         "CR", "NR", "TC", "Z9", "SN", 
         "J9", "JI", "PD", "PY", "VL", 
         "AR", "DI", "SC"]

df_ab = pd.DataFrame(columns=columns)

In [ ]:
starting_index = 1862
data_index = list(range(starting_index, df_re.shape[0]))

for art in tqdm(data_index, desc="looping over df_re"):

    eid = df_re.loc[art, "eid"]
    try:
        ab = AbstractRetrieval(eid, view="FULL")
    except:
        print(f"# {art}: abstract retrieval error")
        continue

    # 1. PT: publication type
    docu_type_ = ab.srctype

    # 2. AU: index name
    index_name_ = [a.indexed_name for a in ab.authors] if ab.authors else [None]

    # 3. AF: author name
    author_name_ = [f"{a.surname}, {a.given_name}" for a in ab.authors] if ab.authors else [None]

    # 4. TI: document title
    docu_title_ = df_re.loc[art, "title"]

    # 5. SO: publication name
    src_title_ = ab.publicationName

    # 5-1. publication abbr.
    src_abb_ = ab.sourcetitle_abbreviation

    # 6. LA : Language
    try:
        language_ = dic_language[ab.language]
    except:
        language_ = "unknown"

    # 7. DT : Document Type
    docu_type_ = ab.subtype

    # 8. DE : Author Keywords
    auth_kw_ = ab.authkeywords

    if not auth_kw_:
        auth_kw_ = 'None'
    else:
        auth_kw_ = '; '.join(auth_kw_)

    # 9. ID : Keyword Plus
    kw_plus_ = 'None'

    # 10. AB : Abstract
    abstract_ = ab.abstract

    # 11. C1 : Author Address
#     if ab.authorgroup:
#         tmp = pd.DataFrame(ab.authorgroup)
#         grouped = tmp.groupby('organization').agg(lambda x: list(x))

#         aff_names = [str(aff) for aff in grouped["affiliation_id"].index.tolist()]
#         aff_ids = grouped["affiliation_id"].tolist()
#         citys = grouped["city"].tolist()
#         countrys = grouped["country"].tolist()
#         auids = grouped["auid"].tolist()
#         indexed_names = grouped["indexed_name"].tolist()

#         address = []
#         for aff_name, aff_id, city, country in zip(aff_names, aff_ids, citys, countrys):
#             if isinstance(aff_id, list):
#                 aff_id, city, country = aff_id[0], city[0], country[0]
#             address.append(f"{aff_id}, {aff_name}, {city}, {country}")

#         addresss_ = list(zip(auids, indexed_names, address))
    if ab.affiliation and ab.authors:
        df_aff = pd.DataFrame(ab.affiliation)
        df_aff["id"] = df_aff["id"].astype(str)
        df_auth = pd.DataFrame(ab.authors)
        df_authgroup = df_auth.groupby("affiliation").agg(list).reset_index()

        df_aff = pd.merge(df_aff, df_authgroup, left_on="id", right_on="affiliation").drop(["affiliation"], axis=1)
        df_aff["address"] = df_aff[['id', 'name', 'city', 'country']].apply(lambda x: ', '.join(x.astype(str)), axis=1)
        addresss_ = df_aff[["auid", "indexed_name", "address"]].values.tolist()
    else:
        addresss_ = []

    # df_aff = pd.DataFrame(ab.authorgroup)
    # grouped = df_aff.groupby("affiliation_id")
    # grouped[["organization", "affiliation_id", "city", "country"]]

    # 12. RP : Reprint Address
    rep_addr_ = "None"

    # 13. EM : E-mail Address
    em_addr_ = "None"

    # 14. CR : Cited References
    refs_ = []
    if ab.references != None:
        tmp = pd.DataFrame(ab.references)
        refcount = int(ab.refcount)

        for i in range(refcount):
            tmp_ = tmp.iloc[i]
            tmp_authors = tmp_['authors']
            if tmp_authors == None:
                tmp_authors = "[Anonymous]"
            tmp_year = tmp_['publicationyear']
            tmp_src = tmp_['sourcetitle']
            tmp_vol = tmp_['volume']
            tmp_page = tmp_['first']
            tmp_doi = tmp_['doi']

            ref = tmp_authors
            for item in [tmp_year, tmp_src, tmp_vol, tmp_page]:
                if item != None:
                    ref = ', '.join([ref, item])
            if tmp_doi != None:
                ref = ref + f", DOI {tmp_doi}"

            if i == 0:
                refs_.append(f"CR {ref}")
            else:
                refs_.append(f"   {ref}")

    # 15. NR : Cited Reference Count
    nr_ = ab.refcount

    # 16. TC : Web of Science Core Collection Times Cited Count
    tc_ = ab.citedby_count

    # 17. Z9 : Total Times Cited Count
    cc_ = tc_

    # 18. U1 : Usage Count (Last 180 Days)
    # 19. U2 : Usage Count (Since 2013)
    # 20. PU : Publisher = ELSEVIER SCI LTD
    # 21. PI : Publisher City = OXFORD
    # 22. PA : Publisher Address = THE BOULEVARD, LANGFORD LANE, KIDLINGTON, OXFORD OX5 1GB, OXON, ENGLAND
    # 23. SN : International Standard Serial Number (ISSN) = 0959-6526
    sn_ = "None" if ab.issn == None else ab.issn

    # 24. EI : Electronic International Standard Serial Number (eISSN) = 1879-1786
    # 25. J9 : 29-Character Source Abbreviation = J CLEAN PROD
    j9_ = ab.sourcetitle_abbreviation if ab.sourcetitle_abbreviation != None else "None"
    j9_ = j9_.upper()

    # 26. JI : ISO Source Abbreviation = J. Clean Prod.
    ji_ = ab.sourcetitle_abbreviation if ab.sourcetitle_abbreviation != None else "None"

    # 27. PD : Publication Date = JUL 1
    month = ab.coverDate.split('-')[1]
    date = ab.coverDate.split('-')[2]
    pd_ = f"{calendar.month_name[int(month)][:3].upper()} {int(date)}"

    # 28. PY : Publication Year = 2020
    py_ = ab.coverDate.split('-')[0]

    # 29. VL : Volumn = 260
    vl_ = ab.volume

    # 30. AR : Article Number = 121059
    ar_ = df_re.loc[art, "article_number"] if df_re.loc[art, "article_number"] != None else "None"

    # 31. DI : Digital Object Identifier = 10.1016/j.jclepro.2020.121059
    doi_ = ab.doi

    # 32. PG : Page Count = 14
    # 33. WC : Web of Science Categories = Green & Sustainable Science & Technology; Engineering, Environmental; Environmental Sciences
    # 34. SC : Research Areas = Science & Technology - Other Topics; Engineering; Environmental Sciences & Ecology
    if ab.subject_areas:        
        tmp = pd.DataFrame(ab.subject_areas)
        tmp_ = tmp["area"].tolist()
        sc_ = "; ".join(tmp_)
    else: 
        sc_ = [None]

    # 35. GA : Document Delivery Number = LL4XH
    # 36. UT : Accession Number = WOS:000531559900003
    # 37. DA : Date this report was generated. = 2020-06-14
    
    # summation
    data=[eid, docu_type_, index_name_, author_name_, docu_title_, 
                                src_title_, src_abb_, language_, docu_type_, auth_kw_, 
                                kw_plus_, abstract_, addresss_, rep_addr_, em_addr_, 
                                refs_, nr_, tc_, cc_, sn_, 
                                j9_, ji_, pd_, py_, vl_, 
                                ar_, doi_, sc_]
    
    df_ab = df_ab.append(dict(zip(columns, data)), ignore_index=True)

looping over df_re:   0%|          | 1/21943 [00:00<2:19:28,  2.62it/s]

# 1862: abstract retrieval error


looping over df_re:   1%|          | 246/21943 [03:01<4:53:13,  1.23it/s]

In [7]:
list_rooftop_eid = df_rooftop["eid"].tolist()
df_ab["is_rooftop"] = df_ab["eid"].apply(lambda x: True if x in list_rooftop_eid else False)

In [8]:
df_ab.to_pickle("./210803/df_ab.pkl")
df_ab.shape

(14385, 29)